In [ ]:

# Required columns (DB schema)
required_columns = [
    "exchange",
    "exchange_token",
    "trading_symbol",
    "name",
    "instrument_type",
    "segment",
    "series",
    "isin",
    "expiry_date",
    "strike_price",
    "lot_size"
]

# Keep only existing columns
existing_columns = [c for c in required_columns if c in df.columns]

# Filter dataframe
final_df = df.loc[:, existing_columns].copy()
final_df    

In [3]:
import duckdb

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\data\Company Fundamentals\corporate_announcements.duckdb"

# CONNECT
con = duckdb.connect(db_path)

# CHECK TABLES
tables = con.execute("SHOW TABLES").fetchall()
print("Tables:", tables)


# CLOSE
con.close()


Tables: [('corporate_announcements',)]


In [5]:
for table in ["corporate_announcements"]:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count}")


corporate_announcements: 0


In [8]:
import requests
import json
import pandas as pd
from datetime import datetime
import websocket
import threading


In [9]:

import requests
AUTH_URL = "https://auth.truedata.in/token"

USERNAME = "trial134"
PASSWORD = "sandy134"


url = "https://auth.truedata.in/token"

payload = {
    "username": USERNAME,
    "password": PASSWORD,
    "grant_type": "password"
}

resp = requests.post(url, data=payload, timeout=10)

print("Status:", resp.status_code)
print(resp.json())



Status: 200
{'access_token': 'tcHLrCSgBKka1VA_3K3IKC0ZkhZPHeW_1DwpxC20IhysZ8xZmZE8GwLtSv7C5uE0LFv3QNbMkOFii28WXO2TR_sRek01KWLBESnozfZoUGgQgn74bGQNtTiIselU59M-StwhJ5x8L17UY9YsBjotpoqtyE9_xtivZr32rm9TPdXEACjEPI9RTil8eFDr_NlKtPwHuJiweQ7q-_weB4uboNDMmvIPGuHYMyFJa2tqBLx6ZdTNO5ilH0DCcHR34wtlGjE6e454DKrUmtbk6tNwNw', 'token_type': 'bearer', 'expires_in': 47097, 'userName': 'trial134', '.expires': 'Wed, 07 Jan 2026 22:30:00 GMT', '.issued': 'Wed, 07 Jan 2026 09:25:02 GMT'}


In [10]:
token = resp.json()["access_token"]

headers = {
    "Authorization": f"Bearer {token}"
}

url = "https://corporate.truedata.in/getdescriptors?response=csv"

r = requests.get(url, headers=headers, timeout=10)

print("Status:", r.status_code)
print(r.text[:500])


Status: 200
id,descriptor,category
1,General,General Announcement/ General Updates
2,Acquisition,Announcement under Regulation 30 (LODR)
3,AGM,Shareholder Meeting / Postal Ballot
5,Allotment,Announcement under Regulation 30 (LODR)
6,Change_in_RTA,Announcement under Regulation 30 (LODR)
7,Amalgamation_Merger_Demerger,Corporate Action
8,Award_of_Order_Receipt_of_Order,Announcement under Regulation 30 (LODR)
9,Board_Meeting,Board Meeting
10,Bonus,Corporate Action
11,Book_Closure,Corporate Action
12,


In [12]:
url = (
    "https://corporate.truedata.in/"
    "getannouncementsforcompaniesrange"
    "?response=json&from=250815 00:00:00&to=250817 23:59:59"
)

r = requests.get(url, headers=headers, timeout=15)

print("Status:", r.status_code)
print(r.text[:500])


Status: 200
{"status":"Success","Records":[["4519411","17-08-2025 23:27:06",532879,"","SSLEL_BSE","Sir Shadi Lal Enterprises Ltd","N","Announcement under Regulation 30 (LODR)-Newspaper Publication","Announcement under Regulation 30 (LODR)-Newspaper Publication","Notice of 91st Annual General Meeting & E-voting information published in the newspaper","Newspaper Publication","General_Announcements","",0,234.4,0],["4519410","17-08-2025 22:44:50",517170,"","EDVENSWA_BSE","Edvenswa Enterprises Ltd","N","Newspape


In [19]:
url = "https://corporate.truedata.in/getSymbolClassification?symbol=ACC&response=json"

r = requests.get(url, headers=headers, timeout=10)

print("Status:", r.status_code)
print(r.json())


Status: 200
{'status': 'Success', 'Records': [{'symbol': 'ACC', 'company': 'ACC LIMITED', 'isin': 'INE012A01025', 'sector': 'Commodities', 'industry': 'Construction Materials', 'group': 'Cement & Cement Products', 'subgroup': 'Cement & Cement Products'}]}


In [13]:
ANNOUNCEMENT_ID = 706710  # example

url = f"https://corporate.truedata.in/announcementfile?id={ANNOUNCEMENT_ID}"

r = requests.get(url, headers=headers, timeout=15)

print("Status:", r.status_code)
print("Content-Type:", r.headers.get("Content-Type"))
print("Content-Length:", len(r.content))


Status: 200
Content-Type: application/pdf
Content-Length: 4947668


In [14]:
import websocket
import json

def on_message(ws, message):
    data = json.loads(message)
    print("\n📢 LIVE ANNOUNCEMENT")
    print("Company:", data.get("CompanyName"))
    print("Symbol NSE:", data.get("Symbol_Nse"))
    print("Headline:", data.get("HeadLine"))
    print("Time:", data.get("TradeDate"))

def on_open(ws):
    print("✅ WebSocket Connected")

def on_error(ws, error):
    print("❌ WebSocket Error:", error)

def on_close(ws):
    print("🔌 WebSocket Closed")

ws_url = f"wss://corp.truedata.in:9092?user={USERNAME}&password={PASSWORD}"

ws = websocket.WebSocketApp(
    ws_url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

ws.run_forever()


✅ WebSocket Connected

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: PTC Industries Ltd
Symbol NSE: PTCIL
Headline: Announcement under Regulation 30 (LODR)-Analyst / Investor Meet - Intimation
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None
❌ WebSocket Error: 
❌ WebSocket Error: on_close() takes 1 positional argument but 3 were given


True

In [15]:
import requests

URL = "https://api.truedata.in/getAllSymbols"

USERNAME = "trial134"
PASSWORD = "sandy134"

PARAMS = {
    "segment": "eq",
    "user": USERNAME,
    "password": PASSWORD
}

response = requests.get(URL, params=PARAMS, timeout=120)

print("HTTP Status:", response.status_code)

if response.status_code != 200:
    raise Exception("Failed to fetch symbols")

data = response.json()

print("Top-level keys:", data.keys())


HTTP Status: 200
Top-level keys: dict_keys(['status', 'Records'])
